In [2]:
import numpy as np
from typing import Tuple
import torch
import pandas as pd

from torch.nn.functional import mse_loss

# 1 Matrix Factorization Using Gradient Descent

In [3]:
def sgd_factorise(A: torch.Tensor, rank: int, num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    m, n = A.shape[0], A.shape[1]
    U = torch.rand(m, rank)
    V = torch.rand(n, rank)

    for epoch in range(num_epochs):
        for r in range(m): 
            for c in range(n):
                e = A[r,c] - U[r,:] @ V[c,:].t()
                U[r, :] = U[r, :] + lr * e * V[c, :]
                V[c, :] = V[c, :] + lr * e * U[r, :]
    return U, V

In [4]:
A = torch.tensor([[0.3374, 0.6005, 0.1735],
                  [3.3359, 0.0492, 1.8374],
                  [2.9407, 0.5301, 2.2620]], dtype=torch.float)



In [5]:
U, V = sgd_factorise(A, 2)
sgd_loss = mse_loss(A, U@V.t(), reduction='sum')
print("SGD reconstruction error: ", sgd_loss.item())

SGD reconstruction error:  0.12241275608539581


In [6]:
print(U@V.t(), mse_loss(A, U@V.t(), reduction='sum'))

tensor([[ 2.2627e-01,  5.3989e-01,  3.5064e-01],
        [ 3.2428e+00, -1.4864e-03,  1.9860e+00],
        [ 3.0449e+00,  5.8703e-01,  2.0958e+00]]) tensor(0.1224)


# 2 Compare your result to truncated SVD

In [7]:
# SGD Solution 
U, V = sgd_factorise(A, 2)
sgd_loss = mse_loss(A, U@V.t(), reduction='sum')

# SVD Solution 
U, S, Vh = torch.linalg.svd(A)
S = torch.diag(S)
A_ = U[:, :2] @ S[:2, :2] @ Vh[:2, :]
svd_loss = mse_loss(A, A_, reduction='sum')
l = (A-A_)**2
l = l.sum()

print("SGD reconstruction loss: ", sgd_loss.item(), "SVD reconstruction loss:", svd_loss.item())

SGD reconstruction loss:  0.12195731699466705 SVD reconstruction loss: 0.12191080302000046


In [8]:
np.sqrt(l)

tensor(0.3492)

In [9]:
np.sqrt(0.12232)

0.3497427626127523

## Explanation


The best rank-k approximation to D in the spectral norm, is given by <br>

$ D_k^* := \sum^k_{i=1} \sigma_i, u_i, v_i^T $ <br>

The Eckart-Young-Mirsky theorm shows that the frobenius norm of the reconstruction error is equal to the square root of the sum of unexplained variances in the singular value matrix. As the frobenious norm is the function minimized by sgd the reconstruction error should be equal to this unexplained variance.

$||D - \hat{D}*||_F = \underset{rank(\hat{D} \leq r)}{min}||D - \hat{D}||_F = \sqrt{\sigma_1^2 + ... + \sigma_r^2} $

given that $\Sigma$ is;

In [10]:
S

tensor([[5.3339, 0.0000, 0.0000],
        [0.0000, 0.6959, 0.0000],
        [0.0000, 0.0000, 0.3492]])

$\sqrt{\sigma_1^2 + ... + \sigma_r^2} = 0.3492$ <br>

The loss function used for sgd solution was the square of the frobenius norm and hence the square root of sgd_loss $\sqrt{0.1219}$ returns the third eigenvalue (0.3492) thus reaching the global minimum.


# 3 Matrix Completion

In [11]:
def sgd_partial_factorise(A: torch.Tensor, M: torch.Tensor, rank: int, num_epochs=1000, lr=0.01) -> Tuple[torch.Tensor, torch.Tensor]:
    m, n = A.shape[0], A.shape[1]
    U = torch.rand(m, rank)
    V = torch.rand(n, rank)
    
    for epoch in range(num_epochs):
        for r in range(m): 
            for c in range(n):
                if M[r,c] == 1:
                    e = A[r,c] - U[r,:] @ V[c,:].t()
                    U[r, :] = U[r, :] + lr * e * V[c, :]
                    V[c, :] = V[c, :] + lr * e * U[r, :]
    return U, V



In [12]:
A_partial = torch.tensor([[0.3374, 0.6005, 0.1735],
                  [0., 0.0492, 1.8374],
                  [2.9407, 0., 2.2620]], dtype=torch.float)

M = torch.tensor([[1,1,1],
                  [0,1,1],
                  [1,0,1]], dtype=torch.int)

In [14]:
U, V = sgd_partial_factorise(A_partial, M, 2)
sgd_loss = mse_loss(A, U@V.t(), reduction='sum')
print(U@V.t(), "SGD reconstruction error: ", sgd_loss.item())

tensor([[0.3869, 0.5572, 0.1269],
        [1.9816, 0.0472, 1.8302],
        [2.9284, 1.0882, 2.2763]]) SGD reconstruction error:  2.152406930923462


## What Does This Tell You ?

The Loss is higher when using the mask. This however is to be expected due to information being missing from matrix A_partial. The loss is small and hence shows that it is possible to impute the values for unknown data based on a linear comibination of other features.

# 4 Movie Recommendation

In [47]:
A = torch.load('data/lab1/ratings.pt')
titles = pd.read_csv('data/lab1/titles.csv', sep=',')

In [48]:
def gd_factorise_masked(A: torch.Tensor, M: torch.Tensor, rank: int, num_epochs: int=1000, lr:float=1e-5) -> Tuple[torch.Tensor, torch.Tensor]:
    U = torch.rand(A.shape[0], rank)
    V = torch.rand(A.shape[1], rank)
    
    for e in range(num_epochs):
        err = (A - U@V.t()) * M
        U += lr * err @ V
        V += lr * err.t() @ U
        
    return U, V
                                                                                                                   

In [53]:
"""
======================== IMPORTANT ======================== 

No information given regarding mask. Hence;

Assumtion: Score of 0. indicates missing value 
"""
U, V = gd_factorise_masked(A, A.ge(0.0001), 5)

In [54]:
A_ = U@V.t()

In [55]:
movie_index = titles.loc[titles['name']=="A Beautiful Mind"].to_numpy()[0,0]
user_index = 4

loss_values = ((A-A_)**2)
unmasked_loss = sum(sum(loss_values*A.ge(0.0001)))


In [56]:
print("User 5 gives A Beautiful Mind a rating of ", A_[movie_index, user_index].item())
print("The sum of unmasked squared errors over the whole matrix is: ", unmasked_loss.item())

User 5 gives A Beautiful Mind a rating of  3.40004563331604
The sum of unmasked squared errors over the whole matrix is:  3935211.0
